In [11]:
import numpy as np
import pandas as pd
import re
import statistics
import rpy2.robjects as robjects
from rpy2.robjects import r
robjects.r.rhome = "../../Downloads/PROCESSING_SCRIPTS.R"
%load_ext rpy2.ipython

In [8]:
r_script_path = "PROCESSING_SCRIPTS.R"
robjects.r.source(r_script_path)

R[write to console]: trying URL 'https://mirrors.nics.utk.edu/cran/bin/macosx/big-sur-arm64/contrib/4.3/readxl_1.4.3.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1548809 bytes (1.5 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to


The downloaded binary packages are in
	/var/folders/6j/xz2lxb2161q8384hcfxc24b40000gn/T//RtmpcgFYfb/downloaded_packages


R[write to console]: trying URL 'https://mirrors.nics.utk.edu/cran/bin/macosx/big-sur-arm64/contrib/4.3/readr_2.1.4.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1986319 bytes (1.9 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to 


The downloaded binary packages are in
	/var/folders/6j/xz2lxb2161q8384hcfxc24b40000gn/T//RtmpcgFYfb/downloaded_packages


R[write to console]: trying URL 'https://mirrors.nics.utk.edu/cran/bin/macosx/big-sur-arm64/contrib/4.3/stringr_1.5.0.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 311937 bytes (304 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to

...

R[write to console]: trying URL 'https://mirrors.nics.utk.edu/cran/bin/macosx/big-sur-arm64/contrib/4.3/dplyr_1.1.3.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1583381 bytes (1.5 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to 


The downloaded binary packages are in
	/var/folders/6j/xz2lxb2161q8384hcfxc24b40000gn/T//RtmpcgFYfb/downloaded_packages


R[write to console]: trying URL 'https://mirrors.nics.utk.edu/cran/bin/macosx/big-sur-arm64/contrib/4.3/tidyr_1.3.0.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1340712 bytes (1.3 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to 


The downloaded binary packages are in
	/var/folders/6j/xz2lxb2161q8384hcfxc24b40000gn/T//RtmpcgFYfb/downloaded_packages
Rows: 10700 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): spelling, IPA, pronunciation

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
================================================================================================================================================================================================================================================`summarise()` has grouped output by 'position', 'phoneme'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'phoneme'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'position', 'phoneme'. You can override
using the `.groups` argument.
`summarise()` has gro

value,[16]
visible,[10]


In [12]:
%%R

# Assuming you've already loaded the data
# using the robjects.r.source("PROCESSING_SCRIPTS.R") as mentioned earlier

# Display the first few rows of the R data frame
head(word_initial_mappings)

# A tibble: 6 × 2
  phoneme grapheme
  <chr>   <chr>   
1 @       a       
2 @       a_e     
3 @       au      
4 5       ai_e    
5 5       ay      
6 5       aye     


In [175]:
df = pd.read_csv("forR_input_updated.csv", dtype=str)
df = df.apply(lambda col: col.str.lower() if col.dtype == 'object' else col)

# df = df[df['STRING'].isin(['all', 'cat', 'con', 'hat', 'slam', 'break', 'shock'])] # USE THIS FOR TESTING (COMMENT LINE BELOW WHEN TESTING)

# df = df.loc[df['syllable2a_p'].isna()]

df

,STRING,syllable1a_p,syllable1a_g,syllable1a_pg,syllable1b_p,syllable1b_g,syllable1b_pg,syllable1c_p,syllable1c_g,syllable1c_pg,...,syllable6c_pg,syllable6d_p,syllable6d_g,syllable6d_pg,syllable6e_p,syllable6e_g,syllable6e_pg,syllable6f_p,syllable6f_g,syllable6f_pg
0,a,eɪ,a,eɪ+a,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a,ə,a,ə+a,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aah,ɑ,aah,ɑ+aah,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aardvark,ɑ,aa,ɑ+aa,r,r,r+r,d,d,d+d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aaron,ɛ,aa,ɛ+aa,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10696,zone,z,z,z+z,oʊ,o,oʊ+o_e,n,n,n+n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10697,zoo,z,z,z+z,u,oo,u+oo,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10698,zoom,z,z,z+z,u,oo,u+oo,m,m,m+m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10699,zounds,z,z,z+z,aʊ,ou,aʊ+ou,n,n,n+n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
#determines the number of variables based on which rows are NaN
def find_syl(row):
    if str(row['syllable2a_p']).lower() == 'nan':
        return 1
    elif str(row['syllable3a_p']).lower() == 'nan':
        return 2
    elif str(row['syllable4a_p']).lower() == 'nan':
        return 3
    elif str(row['syllable5a_p']).lower() == 'nan':
        return 4
    elif str(row['syllable6a_p']).lower() == 'nan':
        return 5
    else:
        return 6

In [177]:
df.insert(1,'complexity', 0)
df.insert(2,'syllables', 0)
df['syllables'] = df.apply(find_syl, axis=1)
df

,STRING,complexity,syllables,syllable1a_p,syllable1a_g,syllable1a_pg,syllable1b_p,syllable1b_g,syllable1b_pg,syllable1c_p,...,syllable6c_pg,syllable6d_p,syllable6d_g,syllable6d_pg,syllable6e_p,syllable6e_g,syllable6e_pg,syllable6f_p,syllable6f_g,syllable6f_pg
0,a,0,1,eɪ,a,eɪ+a,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a,0,1,ə,a,ə+a,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aah,0,1,ɑ,aah,ɑ+aah,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aardvark,0,2,ɑ,aa,ɑ+aa,r,r,r+r,d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aaron,0,2,ɛ,aa,ɛ+aa,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10696,zone,0,1,z,z,z+z,oʊ,o,oʊ+o_e,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10697,zoo,0,1,z,z,z+z,u,oo,u+oo,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10698,zoom,0,1,z,z,z+z,u,oo,u+oo,m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10699,zounds,0,1,z,z,z+z,aʊ,ou,aʊ+ou,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
vowels = ['a', 'e', 'i', 'o', 'u']
syllable_grapheme_to_phoneme_cols_dict = {
    1: {'syllable1a_g': 'syllable1a_p', 
        'syllable1b_g': 'syllable1b_p', 
        'syllable1c_g': 'syllable1c_p', 
        'syllable1d_g': 'syllable1d_p', 
        'syllable1e_g': 'syllable1e_p', 
        'syllable1f_g': 'syllable1f_p'
        },
    2: {'syllable2a_g': 'syllable2a_p', 
        'syllable2b_g': 'syllable2b_p', 
        'syllable2c_g': 'syllable2c_p', 
        'syllable2d_g': 'syllable2d_p', 
        'syllable2e_g': 'syllable2e_p', 
        'syllable2f_g': 'syllable2f_p'
        },
    3: {'syllable3a_g': 'syllable3a_p', 
        'syllable3b_g': 'syllable3b_p', 
        'syllable3c_g': 'syllable3c_p', 
        'syllable3d_g': 'syllable3d_p', 
        'syllable3e_g': 'syllable3e_p', 
        'syllable3f_g': 'syllable3f_p'
        },
    4: {'syllable4a_g': 'syllable4a_p', 
        'syllable4b_g': 'syllable4b_p', 
        'syllable4c_g': 'syllable4c_p', 
        'syllable4d_g': 'syllable4d_p', 
        'syllable4e_g': 'syllable4e_p', 
        'syllable4f_g': 'syllable4f_p'
        },
    5: {'syllable5a_g': 'syllable5a_p', 
        'syllable5b_g': 'syllable5b_p', 
        'syllable5c_g': 'syllable5c_p', 
        'syllable5d_g': 'syllable5d_p', 
        'syllable5e_g': 'syllable5e_p', 
        'syllable5f_g': 'syllable5f_p'
        },
    6: {'syllable6a_g': 'syllable6a_p', 
        'syllable6b_g': 'syllable6b_p', 
        'syllable6c_g': 'syllable6c_p', 
        'syllable6d_g': 'syllable6d_p', 
        'syllable6e_g': 'syllable6e_p', 
        'syllable6f_g': 'syllable6f_p'
        }}

all_syllable_columns = syllable_grapheme_to_phoneme_cols_dict[1] | syllable_grapheme_to_phoneme_cols_dict[2] | syllable_grapheme_to_phoneme_cols_dict[3] | syllable_grapheme_to_phoneme_cols_dict[4] | syllable_grapheme_to_phoneme_cols_dict[5] | syllable_grapheme_to_phoneme_cols_dict[6]

def concatenate_syllable_graphemes(row):
    num_syl = row['syllables']
    syllables_list_g = ['', '', '', '', '', '']
    for curr_syl in range(1, num_syl + 1):
        syllable_g = ''
        for g, p in syllable_grapheme_to_phoneme_cols_dict[curr_syl].items():
            if str(row[g]).lower() != 'nan':
                syllable_g += str(row[g])
            else:
                break
        syllables_list_g[curr_syl - 1] = syllable_g
    return pd.Series(syllables_list_g, index=['syllable1_g', 'syllable2_g', 'syllable3_g', 'syllable4_g', 'syllable5_g', 'syllable6_g'])

def concatenate_syllable_phonemes(row):
    num_syl = row['syllables']
    syllables_list_p = ['', '', '', '', '', '']
    for curr_syl in range(1, num_syl + 1):
        syllable_p = ''
        for g, p in syllable_grapheme_to_phoneme_cols_dict[curr_syl].items():
            if str(row[p]).lower() != 'nan':
                syllable_p += str(row[p])
            else:
                break
        syllables_list_p[curr_syl - 1] = syllable_p
    return pd.Series(syllables_list_p, index=['syllable1_p', 'syllable2_p', 'syllable3_p', 'syllable4_p', 'syllable5_p', 'syllable6_p'])

df[['syllable1_g', 'syllable2_g', 'syllable3_g', 'syllable4_g', 'syllable5_g', 'syllable6_g']] = df.apply(concatenate_syllable_graphemes, axis=1)
df[['syllable1_p', 'syllable2_p', 'syllable3_p', 'syllable4_p', 'syllable5_p', 'syllable6_p']] = df.apply(concatenate_syllable_phonemes, axis=1)
df[['syllable1_g', 'syllable2_g', 'syllable3_g', 'syllable4_g', 'syllable5_g', 'syllable6_g']] = df[['syllable1_g', 'syllable2_g', 'syllable3_g', 'syllable4_g', 'syllable5_g', 'syllable6_g']].replace('', np.nan)
df[['syllable1_p', 'syllable2_p', 'syllable3_p', 'syllable4_p', 'syllable5_p', 'syllable6_p']] = df[['syllable1_p', 'syllable2_p', 'syllable3_p', 'syllable4_p', 'syllable5_p', 'syllable6_p']].replace('', np.nan)

def get_word_phonemes(row):
    word_phoneme = ''
    for g, p in all_syllable_columns.items():
        if str(row[p]).lower() != 'nan':
            word_phoneme += str(row[p])
    return word_phoneme

df.insert(3,'word_p', 0)
df['word_p'] = df.apply(get_word_phonemes, axis=1)

df

,STRING,complexity,syllables,word_p,syllable1a_p,syllable1a_g,syllable1a_pg,syllable1b_p,syllable1b_g,syllable1b_pg,...,syllable3_g,syllable4_g,syllable5_g,syllable6_g,syllable1_p,syllable2_p,syllable3_p,syllable4_p,syllable5_p,syllable6_p
0,a,0,1,eɪ,eɪ,a,eɪ+a,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,eɪ,NaN,NaN,NaN,NaN,NaN
1,a,0,1,ə,ə,a,ə+a,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ə,NaN,NaN,NaN,NaN,NaN
2,aah,0,1,ɑ,ɑ,aah,ɑ+aah,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ɑ,NaN,NaN,NaN,NaN,NaN
3,aardvark,0,2,ɑrdvɑrk,ɑ,aa,ɑ+aa,r,r,r+r,...,NaN,NaN,NaN,NaN,ɑrd,vɑrk,NaN,NaN,NaN,NaN
4,aaron,0,2,ɛrən,ɛ,aa,ɛ+aa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ɛ,rən,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10696,zone,0,1,zoʊn,z,z,z+z,oʊ,o,oʊ+o_e,...,NaN,NaN,NaN,NaN,zoʊn,NaN,NaN,NaN,NaN,NaN
10697,zoo,0,1,zu,z,z,z+z,u,oo,u+oo,...,NaN,NaN,NaN,NaN,zu,NaN,NaN,NaN,NaN,NaN
10698,zoom,0,1,zum,z,z,z+z,u,oo,u+oo,...,NaN,NaN,NaN,NaN,zum,NaN,NaN,NaN,NaN,NaN
10699,zounds,0,1,zaʊndz,z,z,z+z,aʊ,ou,aʊ+ou,...,NaN,NaN,NaN,NaN,zaʊndz,NaN,NaN,NaN,NaN,NaN


In [179]:
def find_word_onset_and_rime_graphemes(row):
    word = str(row['STRING'])
    onset = ''
    rime = ''
    index = 0
    while index < len(word) and word[index] not in vowels:
        onset += word[index]
        index += 1
    rime = word[index:]
    return pd.Series([onset, rime], index=['word_onset_g', 'word_rime_g'])

def find_syllable_onset_and_rime_graphemes(row):
    num_syl = row['syllables']
    syl_onsets = ['', '', '', '', '', '']
    syl_rimes = ['', '', '', '', '', '']
    for curr_syl in range(1, num_syl + 1):
        curr_syllable_g = row['syllable' + str(curr_syl) + '_g']
        onset = ''
        rime = ''
        index = 0
        while index < len(str(curr_syllable_g)) and str(curr_syllable_g)[index] not in vowels:
            onset += str(curr_syllable_g)[index]
            index += 1
        rime = str(curr_syllable_g)[index:]
        syl_onsets[curr_syl - 1] = onset
        syl_rimes[curr_syl - 1] = rime
    syl_onsets_and_rimes = syl_onsets + syl_rimes
    return pd.Series(syl_onsets_and_rimes, index=['syllable1_onset_g', 'syllable2_onset_g', 'syllable3_onset_g', 'syllable4_onset_g', 'syllable5_onset_g', 'syllable6_onset_g', 'syllable1_rime_g', 'syllable2_rime_g', 'syllable3_rime_g', 'syllable4_rime_g', 'syllable5_rime_g', 'syllable6_rime_g'])

df[['syllable1_onset_g', 'syllable2_onset_g', 'syllable3_onset_g', 'syllable4_onset_g', 'syllable5_onset_g', 'syllable6_onset_g', 'syllable1_rime_g', 'syllable2_rime_g', 'syllable3_rime_g', 'syllable4_rime_g', 'syllable5_rime_g', 'syllable6_rime_g']] = df.apply(find_syllable_onset_and_rime_graphemes, axis=1)
df[['syllable1_onset_g', 'syllable2_onset_g', 'syllable3_onset_g', 'syllable4_onset_g', 'syllable5_onset_g', 'syllable6_onset_g', 'syllable1_rime_g', 'syllable2_rime_g', 'syllable3_rime_g', 'syllable4_rime_g', 'syllable5_rime_g', 'syllable6_rime_g']] = df[['syllable1_onset_g', 'syllable2_onset_g', 'syllable3_onset_g', 'syllable4_onset_g', 'syllable5_onset_g', 'syllable6_onset_g', 'syllable1_rime_g', 'syllable2_rime_g', 'syllable3_rime_g', 'syllable4_rime_g', 'syllable5_rime_g', 'syllable6_rime_g']].replace('', np.nan)


def find_word_onset_phonemes(row):
    onset = find_word_onset_and_rime_graphemes(row)[0]
    onset_phonemes = ''
    for grapheme_col in all_syllable_columns:
        curr_grapheme = row[grapheme_col]
        if str(curr_grapheme).lower() == 'nan':
            return onset_phonemes
        start_index = onset.find(curr_grapheme) 
        # shouldn't come here, but just in case
        if start_index == -1:
            return onset_phonemes
        end_index = start_index + len(curr_grapheme)
        onset_phonemes += row[all_syllable_columns[grapheme_col]]
        if end_index >= len(onset):
            return onset_phonemes
        
def find_syllable_onset_phonemes(row):
    syllable_onset_phonemes = []
    syllable_onset_and_rime_graphemes = find_syllable_onset_and_rime_graphemes(row)
    syllable_onset_graphemes = [syllable_onset_and_rime_graphemes[0], syllable_onset_and_rime_graphemes[1], syllable_onset_and_rime_graphemes[2], syllable_onset_and_rime_graphemes[3], syllable_onset_and_rime_graphemes[4], syllable_onset_and_rime_graphemes[5]]
    for i in range(len(syllable_onset_graphemes)):
        onset = syllable_onset_graphemes[i]
        onset_phonemes = ''
        for grapheme_col in syllable_grapheme_to_phoneme_cols_dict[i + 1]:
            curr_grapheme = row[grapheme_col]
            if str(curr_grapheme).lower() == 'nan':
                break
            start_index = onset.find(curr_grapheme) 
            # shouldn't come here, but just in case
            if start_index == -1:
                break
            end_index = start_index + len(curr_grapheme)
            onset_phonemes += row[all_syllable_columns[grapheme_col]]
            if end_index >= len(onset):
                break
        syllable_onset_phonemes.append(onset_phonemes)
    return pd.Series(syllable_onset_phonemes, index=['syllable1_onset_p', 'syllable2_onset_p', 'syllable3_onset_p', 'syllable4_onset_p', 'syllable5_onset_p', 'syllable6_onset_p'])
        
df[['syllable1_onset_p', 'syllable2_onset_p', 'syllable3_onset_p', 'syllable4_onset_p', 'syllable5_onset_p', 'syllable6_onset_p']] = df.apply(find_syllable_onset_phonemes, axis=1)
df[['syllable1_onset_p', 'syllable2_onset_p', 'syllable3_onset_p', 'syllable4_onset_p', 'syllable5_onset_p', 'syllable6_onset_p']] = df[['syllable1_onset_p', 'syllable2_onset_p', 'syllable3_onset_p', 'syllable4_onset_p', 'syllable5_onset_p', 'syllable6_onset_p']].replace('', np.nan)


def find_word_rime_phonemes(row):
    rime = find_word_onset_and_rime_graphemes(row)[1]
    rime_phonemes = ''
    starting_char_found = False
    for grapheme_col in all_syllable_columns:
        curr_grapheme = row[grapheme_col]
        if str(curr_grapheme).lower() == 'nan':
            break
        start_index = rime.find(curr_grapheme) 
        # shouldn't come here, but just in case
        if start_index == -1 and starting_char_found:
            break
        if start_index != -1:
            starting_char_found = True
            end_index = start_index + len(curr_grapheme)
            rime_phonemes += row[all_syllable_columns[grapheme_col]]
            if end_index >= len(rime):
                break
    for grapheme_col, phoneme_col in list(all_syllable_columns.items())[6:]:
        curr_grapheme = row[grapheme_col]
        if str(curr_grapheme).lower() != 'nan':
            rime_phonemes += str(row[phoneme_col])
    return rime_phonemes

def find_syllable_rime_phonemes(row):
    syllable_rime_phonemes = []
    syllable_onset_and_rime_graphemes = find_syllable_onset_and_rime_graphemes(row)
    syllable_rime_graphemes = [syllable_onset_and_rime_graphemes[6], syllable_onset_and_rime_graphemes[7], syllable_onset_and_rime_graphemes[8], syllable_onset_and_rime_graphemes[9], syllable_onset_and_rime_graphemes[10], syllable_onset_and_rime_graphemes[11]]
    for i in range(len(syllable_rime_graphemes)):
        rime = syllable_rime_graphemes[i]
        rime_phonemes = ''
        starting_char_found = False
        for grapheme_col in syllable_grapheme_to_phoneme_cols_dict[i + 1]:
            curr_grapheme = row[grapheme_col]
            if str(curr_grapheme).lower() == 'nan':
                break
            start_index = rime.find(curr_grapheme) 
            # shouldn't come here, but just in case
            if start_index == -1 and starting_char_found:
                break
            if start_index != -1:
                starting_char_found = True
                end_index = start_index + len(curr_grapheme)
                rime_phonemes += row[all_syllable_columns[grapheme_col]]
                if end_index >= len(rime):
                    break
        syllable_rime_phonemes.append(rime_phonemes)
    return pd.Series(syllable_rime_phonemes, index=['syllable1_rime_p', 'syllable2_rime_p', 'syllable3_rime_p', 'syllable4_rime_p', 'syllable5_rime_p', 'syllable6_rime_p'])

df[['syllable1_rime_p', 'syllable2_rime_p', 'syllable3_rime_p', 'syllable4_rime_p', 'syllable5_rime_p', 'syllable6_rime_p']] = df.apply(find_syllable_rime_phonemes, axis=1)
df[['syllable1_rime_p', 'syllable2_rime_p', 'syllable3_rime_p', 'syllable4_rime_p', 'syllable5_rime_p', 'syllable6_rime_p']] = df[['syllable1_rime_p', 'syllable2_rime_p', 'syllable3_rime_p', 'syllable4_rime_p', 'syllable5_rime_p', 'syllable6_rime_p']].replace('', np.nan)


df['word_onset_p'] = df.apply(find_word_onset_phonemes, axis=1)
df['word_onset_p'] = df['word_onset_p'].replace('', np.nan)

df['word_rime_p'] = df.apply(find_word_rime_phonemes, axis=1)
df['word_rime_p'] = df['word_rime_p'].replace('', np.nan)

df[['word_onset_g', 'word_rime_g']] = df.apply(find_word_onset_and_rime_graphemes, axis=1)
df['word_onset_g'] = df['word_onset_g'].replace('', np.nan)
df['word_rime_g'] = df['word_rime_g'].replace('', np.nan)

df

,STRING,complexity,syllables,word_p,syllable1a_p,syllable1a_g,syllable1a_pg,syllable1b_p,syllable1b_g,syllable1b_pg,...,syllable1_rime_p,syllable2_rime_p,syllable3_rime_p,syllable4_rime_p,syllable5_rime_p,syllable6_rime_p,word_onset_p,word_rime_p,word_onset_g,word_rime_g
0,a,0,1,eɪ,eɪ,a,eɪ+a,NaN,NaN,NaN,...,eɪ,NaN,NaN,NaN,NaN,NaN,NaN,eɪ,NaN,a
1,a,0,1,ə,ə,a,ə+a,NaN,NaN,NaN,...,ə,NaN,NaN,NaN,NaN,NaN,NaN,ə,NaN,a
2,aah,0,1,ɑ,ɑ,aah,ɑ+aah,NaN,NaN,NaN,...,ɑ,NaN,NaN,NaN,NaN,NaN,NaN,ɑ,NaN,aah
3,aardvark,0,2,ɑrdvɑrk,ɑ,aa,ɑ+aa,r,r,r+r,...,ɑrd,ɑrk,NaN,NaN,NaN,NaN,NaN,ɑrdvɑrk,NaN,aardvark
4,aaron,0,2,ɛrən,ɛ,aa,ɛ+aa,NaN,NaN,NaN,...,ɛ,ən,NaN,NaN,NaN,NaN,NaN,ɛrən,NaN,aaron
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10696,zone,0,1,zoʊn,z,z,z+z,oʊ,o,oʊ+o_e,...,oʊn,NaN,NaN,NaN,NaN,NaN,z,oʊn,z,one
10697,zoo,0,1,zu,z,z,z+z,u,oo,u+oo,...,u,NaN,NaN,NaN,NaN,NaN,z,u,z,oo
10698,zoom,0,1,zum,z,z,z+z,u,oo,u+oo,...,um,NaN,NaN,NaN,NaN,NaN,z,um,z,oom
10699,zounds,0,1,zaʊndz,z,z,z+z,aʊ,ou,aʊ+ou,...,aʊndz,NaN,NaN,NaN,NaN,NaN,z,aʊndz,z,ounds


In [180]:
def find_word_coda_graphemes(row):
    reversed_word = str(row['STRING'])[::-1] #reverse string to get last letters first
    num_syl = row['syllables']
    grapheme = ''
    for letter in reversed_word:
        if letter in vowels:
            if ((letter == 'e') & (num_syl > 1)): #silent e exception, ending in vowels means no coda
                grapheme = grapheme + letter
            else:
                break
        else:
            grapheme = grapheme + letter
    return grapheme[::-1]

df['word_coda_g'] = df.apply(find_word_coda_graphemes, axis=1)
df['word_coda_g'] = df['word_coda_g'].replace('', np.nan)


def find_word_coda_phonemes(row):
    coda = find_word_coda_graphemes(row)
    num_syl = row['syllables']
    coda_phonemes = ''
    starting_char_found = False
    for grapheme_col in syllable_grapheme_to_phoneme_cols_dict[num_syl]:
        curr_grapheme = row[grapheme_col]
        if str(curr_grapheme).lower() == 'nan':
            return coda_phonemes
        start_index = coda.find(curr_grapheme) 
        # shouldn't come here, but just in case
        if start_index == -1 and starting_char_found:
            return coda_phonemes
        if start_index != -1:
            starting_char_found = True
            end_index = start_index + len(curr_grapheme)
            coda_phonemes += row[all_syllable_columns[grapheme_col]]
            if end_index >= len(coda):
                return coda_phonemes
            
df['word_coda_p'] = df.apply(find_word_coda_phonemes, axis=1)
df['word_coda_p'] = df['word_coda_p'].replace('', np.nan)

df

,STRING,complexity,syllables,word_p,syllable1a_p,syllable1a_g,syllable1a_pg,syllable1b_p,syllable1b_g,syllable1b_pg,...,syllable3_rime_p,syllable4_rime_p,syllable5_rime_p,syllable6_rime_p,word_onset_p,word_rime_p,word_onset_g,word_rime_g,word_coda_g,word_coda_p
0,a,0,1,eɪ,eɪ,a,eɪ+a,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,eɪ,NaN,a,NaN,NaN
1,a,0,1,ə,ə,a,ə+a,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ə,NaN,a,NaN,NaN
2,aah,0,1,ɑ,ɑ,aah,ɑ+aah,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ɑ,NaN,aah,h,NaN
3,aardvark,0,2,ɑrdvɑrk,ɑ,aa,ɑ+aa,r,r,r+r,...,NaN,NaN,NaN,NaN,NaN,ɑrdvɑrk,NaN,aardvark,rk,rk
4,aaron,0,2,ɛrən,ɛ,aa,ɛ+aa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ɛrən,NaN,aaron,n,n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10696,zone,0,1,zoʊn,z,z,z+z,oʊ,o,oʊ+o_e,...,NaN,NaN,NaN,NaN,z,oʊn,z,one,NaN,NaN
10697,zoo,0,1,zu,z,z,z+z,u,oo,u+oo,...,NaN,NaN,NaN,NaN,z,u,z,oo,NaN,NaN
10698,zoom,0,1,zum,z,z,z+z,u,oo,u+oo,...,NaN,NaN,NaN,NaN,z,um,z,oom,m,m
10699,zounds,0,1,zaʊndz,z,z,z+z,aʊ,ou,aʊ+ou,...,NaN,NaN,NaN,NaN,z,aʊndz,z,ounds,nds,ndz


In [183]:
def extract_phoneme_and_grapheme_values(cols_dict):
    phoneme_values = []
    grapheme_values = []
    for g, p in cols_dict.items():
        phoneme_values += df[p].to_list()
        grapheme_values += df[g].to_list()
    return phoneme_values, grapheme_values
    
pg_df = pd.DataFrame()
all_phonemes, all_graphemes = extract_phoneme_and_grapheme_values(all_syllable_columns)
pg_df['phoneme'] = all_phonemes
pg_df['grapheme'] = all_graphemes
pg_df = pg_df.dropna()
    
pg_df

,phoneme,grapheme
0,eɪ,a
1,ə,a
2,ɑ,aah
3,ɑ,aa
4,ɛ,aa
...,...,...
337547,n,n
339415,i,ie
339416,i,y
341880,n,n


In [184]:
syllable_pg_df = pd.DataFrame()

syllable_grapheme_phoneme_cols = {'syllable1_g':'syllable1_p',
                                  'syllable2_g':'syllable2_p',
                                  'syllable3_g':'syllable3_p',
                                  'syllable4_g':'syllable4_p',
                                  'syllable5_g':'syllable5_p',
                                  'syllable6_g':'syllable6_p'} 
    
all_phonemes, all_graphemes = extract_phoneme_and_grapheme_values(syllable_grapheme_phoneme_cols)
syllable_pg_df['syllable_phoneme'] = all_phonemes
syllable_pg_df['syllable_grapheme'] = all_graphemes
syllable_pg_df = syllable_pg_df.dropna()
    
syllable_pg_df

,syllable_phoneme,syllable_grapheme
0,eɪ,a
1,ə,a
2,ɑ,aah
3,ɑrd,aard
4,ɛ,aa
...,...,...
59321,ən,an
60105,ə,a
61189,tiz,ties
61190,ti,ty


In [185]:
word_onset_pg_df = pd.DataFrame()

word_onset_grapheme_phoneme_cols = {'word_onset_g':'word_onset_p'} 
    
all_phonemes, all_graphemes = extract_phoneme_and_grapheme_values(word_onset_grapheme_phoneme_cols)
word_onset_pg_df['word_onset_phoneme'] = all_phonemes
word_onset_pg_df['word_onset_grapheme'] = all_graphemes
word_onset_pg_df = word_onset_pg_df.dropna()
    
word_onset_pg_df

,word_onset_phoneme,word_onset_grapheme
606,b,b
607,b,b
608,b,b
609,b,b
610,b,b
...,...,...
10696,z,z
10697,z,z
10698,z,z
10699,z,z


In [186]:
word_rime_pg_df = pd.DataFrame()

word_rime_grapheme_phoneme_cols = {'word_rime_g':'word_rime_p'} 
    
all_phonemes, all_graphemes = extract_phoneme_and_grapheme_values(word_rime_grapheme_phoneme_cols)
word_rime_pg_df['word_rime_phoneme'] = all_phonemes
word_rime_pg_df['word_rime_grapheme'] = all_graphemes
word_rime_pg_df = word_rime_pg_df.dropna()
    
word_rime_pg_df

,word_rime_phoneme,word_rime_grapheme
0,eɪ,a
1,ə,a
2,ɑ,aah
3,ɑrdvɑrk,aardvark
4,ɛrən,aaron
...,...,...
10696,oʊn,one
10697,u,oo
10698,um,oom
10699,aʊndz,ounds


In [189]:
syllable_onset_pg_df = pd.DataFrame()

syllable_onset_grapheme_phoneme_cols = {1: {'syllable1_onset_g':'syllable1_onset_p'},
                                        2: {'syllable2_onset_g':'syllable2_onset_p'},
                                        3: {'syllable3_onset_g':'syllable3_onset_p'},
                                        4: {'syllable4_onset_g':'syllable4_onset_p'},
                                        5: {'syllable5_onset_g':'syllable5_onset_p'},
                                        6: {'syllable6_onset_g':'syllable6_onset_p'}}

all_phonemes, all_graphemes, syllables_count = [], [], []
for syllable_num in syllable_onset_grapheme_phoneme_cols.keys():
    curr_phonemes, curr_graphemes = extract_phoneme_and_grapheme_values(syllable_onset_grapheme_phoneme_cols[syllable_num])
    curr_syllables_count = [syllable_num] * len(curr_phonemes)
    all_phonemes += curr_phonemes
    all_graphemes += curr_graphemes
    syllables_count += curr_syllables_count

syllable_onset_pg_df['syllable_onset_phoneme'] = all_phonemes
syllable_onset_pg_df['syllable_onset_grapheme'] = all_graphemes
syllable_onset_pg_df['syllable'] = syllables_count
syllable_onset_pg_df = syllable_onset_pg_df.dropna()

syllable_onset_pg_df

,syllable_onset_phoneme,syllable_onset_grapheme,syllable
606,b,b,1
607,b,b,1
608,b,b,1
609,b,b,1
610,b,b,1
...,...,...,...
58128,ti,ty,6
58286,si,sy,6
58440,ti,ty,6
61189,t,t,6


In [191]:
syllable_rime_pg_df = pd.DataFrame()

syllable_onset_grapheme_phoneme_cols = {1: {'syllable1_rime_g':'syllable1_rime_p'},
                                        2: {'syllable2_rime_g':'syllable2_rime_p'},
                                        3: {'syllable3_rime_g':'syllable3_rime_p'},
                                        4: {'syllable4_rime_g':'syllable4_rime_p'},
                                        5: {'syllable5_rime_g':'syllable5_rime_p'},
                                        6: {'syllable6_rime_g':'syllable6_rime_p'}}

all_phonemes, all_graphemes, syllables_count = [], [], []
for syllable_num in syllable_onset_grapheme_phoneme_cols.keys():
    curr_phonemes, curr_graphemes = extract_phoneme_and_grapheme_values(syllable_onset_grapheme_phoneme_cols[syllable_num])
    curr_syllables_count = [syllable_num] * len(curr_phonemes)
    all_phonemes += curr_phonemes
    all_graphemes += curr_graphemes
    syllables_count += curr_syllables_count

syllable_rime_pg_df['syllable_rime_phoneme'] = all_phonemes
syllable_rime_pg_df['syllable_rime_grapheme'] = all_graphemes
syllable_rime_pg_df['syllable'] = syllables_count
syllable_rime_pg_df = syllable_rime_pg_df.dropna()

syllable_rime_pg_df

,syllable_rime_phoneme,syllable_rime_grapheme,syllable
0,eɪ,a,1
1,ə,a,1
2,ɑ,aah,1
3,ɑrd,aard,1
4,ɛ,aa,1
...,...,...,...
58221,ən,an,6
59321,ən,an,6
60105,ə,a,6
61189,iz,ies,6


In [194]:
def conditional_prob_p_to_g(p, g):
    p_g_instances = len((pg_df.loc[(pg_df['phoneme'] == p) & (pg_df['grapheme'] == g)]).index)
    p_instances = len((pg_df.loc[(pg_df['phoneme'] == p)]).index)
    return p_g_instances / p_instances

def conditional_prob_g_to_p(g, p):
    p_g_instances = len((pg_df.loc[(pg_df['phoneme'] == p) & (pg_df['grapheme'] == g)]).index)
    g_instances = len((pg_df.loc[(pg_df['grapheme'] == g)]).index)
    return p_g_instances / g_instances

def p_plus_g_frequency(p, g):
    p_g_instances = len((pg_df.loc[(pg_df['phoneme'] == p) & (pg_df['grapheme'] == g)]).index)
    all_instances = len(pg_df.index)
    return p_g_instances / all_instances

print("p->g for 'ɔ' to 'a':", conditional_prob_p_to_g('ɔ', 'a'))
print("g->p for 'a' to 'ɔ':", conditional_prob_g_to_p('a', 'ɔ'))
print("ɔ+a frequency:", p_plus_g_frequency('ɔ', 'a'))

p->g for 'ɔ' to 'a': 0.17592592592592593
g->p for 'a' to 'ɔ': 0.03171953255425709
ɔ+a frequency: 0.002335217895143449


In [195]:
# ONSET CONDITIONAL PROBABILITIES
def onset_conditional_prob_p_to_g(p, g, option): # options: first_syllable_only, all_syllables, non_first_syllable
    if option == 'first_syllable_only':
        p_g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['grapheme'] == g) & (syllable_onset_pg_df['syllable'] == 1)]).index)
        p_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['syllable'] == 1)]).index)
        return p_g_instances / p_instances
    elif option == 'all_syllables':
        p_g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['grapheme'] == g)]).index)
        p_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p)]).index)
        return p_g_instances / p_instances
    elif option == 'non_first_syllable':
        p_g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['grapheme'] == g) & (syllable_onset_pg_df['syllable'] != 1)]).index)
        p_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['syllable'] != 1)]).index)
        return p_g_instances / p_instances

def onset_conditional_prob_g_to_p(g, p, option): # options: first_syllable_only, all_syllables, non_first_syllable
    if option == 'first_syllable_only':
        p_g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['grapheme'] == g) & (syllable_onset_pg_df['syllable'] == 1)]).index)
        g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['grapheme'] == g) & (syllable_onset_pg_df['syllable'] == 1)]).index)
        return p_g_instances / g_instances
    elif option == 'all_syllables':
        p_g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['grapheme'] == g)]).index)
        g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['grapheme'] == g)]).index)
        return p_g_instances / g_instances
    elif option == 'non_first_syllable':
        p_g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['grapheme'] == g) & (syllable_onset_pg_df['syllable'] != 1)]).index)
        g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['grapheme'] == g) & (syllable_onset_pg_df['syllable'] != 1)]).index)
        return p_g_instances / g_instances
    
def onset_p_plus_g_frequency(p, g, option): # options: first_syllable_only, all_syllables, non_first_syllable
    if option == 'first_syllable_only':
        p_g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['grapheme'] == g) & (syllable_onset_pg_df['syllable'] == 1)]).index)
        all_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['syllable'] != 1)]).index)
        return p_g_instances / all_instances
    elif option == 'all_syllables':
        p_g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['grapheme'] == g)]).index)
        all_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['syllable'] != 1)]).index)
        return p_g_instances / all_instances
    elif option == 'non_first_syllable':
        p_g_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['phoneme'] == p) & (syllable_onset_pg_df['grapheme'] == g) & (syllable_onset_pg_df['syllable'] != 1)]).index)
        all_instances = len((syllable_onset_pg_df.loc[(syllable_onset_pg_df['syllable'] != 1)]).index)
        return p_g_instances / all_instances
    

# RIME CONDITIONAL PROBABILITIES
def rime_conditional_prob_p_to_g(p, g, option): # options: words, syllables
    if option == 'words':
        p_g_instances = len((word_rime_pg_df.loc[(word_rime_pg_df['phoneme'] == p) & (word_rime_pg_df['grapheme'] == g)]).index)
        p_instances = len((word_rime_pg_df.loc[(word_rime_pg_df['phoneme'] == p)]).index)
        return p_g_instances / p_instances
    elif option == 'syllables':
        p_g_instances = len((syllable_rime_pg_df.loc[(syllable_rime_pg_df['phoneme'] == p) & (syllable_rime_pg_df['grapheme'] == g)]).index)
        p_instances = len((syllable_rime_pg_df.loc[(syllable_rime_pg_df['phoneme'] == p)]).index)
        return p_g_instances / p_instances

def rime_conditional_prob_g_to_p(g, p, option): # options: words, syllables
    if option == 'words':
        p_g_instances = len((word_rime_pg_df.loc[(word_rime_pg_df['phoneme'] == p) & (word_rime_pg_df['grapheme'] == g)]).index)
        g_instances = len((word_rime_pg_df.loc[(word_rime_pg_df['grapheme'] == g)]).index)
        return p_g_instances / g_instances
    elif option == 'syllables':
        p_g_instances = len((syllable_rime_pg_df.loc[(syllable_rime_pg_df['phoneme'] == p) & (syllable_rime_pg_df['grapheme'] == g)]).index)
        g_instances = len((syllable_rime_pg_df.loc[(syllable_rime_pg_df['grapheme'] == g)]).index)
        return p_g_instances / g_instances

def rime_p_plus_g_frequency(p, g, option): # options: words, syllables
    if option == 'words':
        p_g_instances = len((word_rime_pg_df.loc[(word_rime_pg_df['phoneme'] == p) & (word_rime_pg_df['grapheme'] == g)]).index)
        all_instances = len(word_rime_pg_df.index)
        return p_g_instances / all_instances
    elif option == 'syllables':
        p_g_instances = len((syllable_rime_pg_df.loc[(syllable_rime_pg_df['phoneme'] == p) & (syllable_rime_pg_df['grapheme'] == g)]).index)
        all_instances = len(syllable_rime_pg_df.index)
        return p_g_instances / all_instances

In [196]:
# p_input = input("Enter phoneme:\n")
# g_input = input("Enter grapheme:\n")

# print(conditional_prob_p_to_g(p_input, g_input))
# print(conditional_prob_g_to_p(g_input, p_input))
# print(p_plus_g_frequency(p_input, g_input))

# option = input("OPTIONS:\n1 for p->g conditional probability.\n2 for g->p conditional probability.\n3 for p+g frequency.\nEnter option:")


In [197]:
# if option == '1':
#     print(conditional_prob_p_to_g(p_input, g_input))
# elif option == '2':
#     print(conditional_prob_g_to_p(g_input, p_input))
# elif option == '3':
#     print(p_plus_g_frequency(p_input, g_input))

In [198]:
# P->G PHONOGRAPHEME CONSISTENCY

def calculate_p_to_g_phonographeme_probs(word):
    word = word.lower()
    p_to_g_probs = []
    if word in df['STRING'].values:
        num_syl = df.loc[df['STRING'] == word, 'syllables'].iloc[0]
        word_phonemes = []
        word_graphemes = []
        for i in range(1, num_syl + 1):
            for g, p in syllable_grapheme_to_phoneme_cols_dict[i].items():
                word_phonemes.append(df.loc[df['STRING'] == word, p].iloc[0])
                word_graphemes.append(df.loc[df['STRING'] == word, g].iloc[0])
        word_phonemes = [x for x in word_phonemes if str(x) != 'nan']
        word_graphemes = [x for x in word_graphemes if str(x) != 'nan']
        for index in range(len(word_phonemes)):
            p_to_g_probs.append(conditional_prob_p_to_g(word_phonemes[index], word_graphemes[index]))
    return p_to_g_probs
    
def p_to_g_phonographeme_consistency_median(word):
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0:
        return statistics.median(p_to_g_probs)
    return None

def p_to_g_phonographeme_consistency_mean(word):
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0:
        return statistics.mean(p_to_g_probs)
    return None

def p_to_g_phonographeme_consistency_max(word):
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0:
        return max(p_to_g_probs)
    return None

def p_to_g_phonographeme_consistency_min(word):
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0:
        return min(p_to_g_probs)
    return None

def p_to_g_phonographeme_consistency_distribution_quantile(word, q): # e.g. q = 0.25 for Q1, q = 0.7 for 70th percentile
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0 and q >= 0 and q <= 100:
        return np.quantile(p_to_g_probs, q)
    return None

def p_to_g_phonographeme_consistency_IQR(word):
    return p_to_g_phonographeme_consistency_distribution_quantile(word, 0.75) - p_to_g_phonographeme_consistency_distribution_quantile(word, 0.25)

def p_to_g_phonographeme_consistency_range(word):
    return p_to_g_phonographeme_consistency_max(word) - p_to_g_phonographeme_consistency_min(word)

In [ ]:
word_input = input("P->G PHONOGRAPHEME CONSISTENCY\nEnter word:\n")
if word_input.lower() not in df['STRING'].values:
    print("Word not found. Exiting.")
else:
    print("MEDIAN:", p_to_g_phonographeme_consistency_median(word_input))
    print("MEAN:", p_to_g_phonographeme_consistency_mean(word_input))
    print("MAX:", p_to_g_phonographeme_consistency_max(word_input))
    print("MIN:", p_to_g_phonographeme_consistency_min(word_input))
    print("Q1:", p_to_g_phonographeme_consistency_distribution_quantile(word_input, 0.25))
    print("IQR:", p_to_g_phonographeme_consistency_IQR(word_input))
    print("RANGE:", p_to_g_phonographeme_consistency_range(word_input))
    # p_to_g_photographeme_consistency_option = input("OPTIONS:\n1 for median.\n2 for mean.\n3 for max.\n4 for min.\nEnter option:")
    
    # if p_to_g_photographeme_consistency_option == '1':
    #     print(p_to_g_phonographeme_consistency_median(word_input))
    # elif p_to_g_photographeme_consistency_option == '2':
    #     print(p_to_g_phonographeme_consistency_mean(word_input))
    # elif p_to_g_photographeme_consistency_option == '3':
    #     print(p_to_g_phonographeme_consistency_max(word_input))
    # elif p_to_g_photographeme_consistency_option == "4":
    #     print(p_to_g_phonographeme_consistency_min(word_input))



MEDIAN: 0.14383735705209658
MEAN: 0.14383735705209658
MAX: 0.2
MIN: 0.08767471410419314
Q1: 0.11575603557814486
IQR: 0.05616264294790345
RANGE: 0.11232528589580687


In [199]:
# G->P PHONOGRAPHEME CONSISTENCY

def calculate_g_to_p_phonographeme_probs(word):
    word = word.lower()
    g_to_p_probs = []
    if word in df['STRING'].values:
        num_syl = df.loc[df['STRING'] == word, 'syllables'].iloc[0]
        word_phonemes = []
        word_graphemes = []
        for i in range(1, num_syl + 1):
            for g, p in syllable_grapheme_to_phoneme_cols_dict[i].items():
                word_phonemes.append(df.loc[df['STRING'] == word, p].iloc[0])
                word_graphemes.append(df.loc[df['STRING'] == word, g].iloc[0])
        word_phonemes = [x for x in word_phonemes if str(x) != 'nan']
        word_graphemes = [x for x in word_graphemes if str(x) != 'nan']
        for index in range(len(word_phonemes)):
            g_to_p_probs.append(conditional_prob_p_to_g(word_graphemes[index], word_phonemes[index]))
    return g_to_p_probs
    
def g_to_p_phonographeme_consistency_median(word):
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0:
        return statistics.median(g_to_p_probs)
    return None

def g_to_p_phonographeme_consistency_mean(word):
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0:
        return statistics.mean(g_to_p_probs)
    return None

def g_to_p_phonographeme_consistency_max(word):
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0:
        return max(g_to_p_probs)
    return None

def g_to_p_phonographeme_consistency_min(word):
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0:
        return min(g_to_p_probs)
    return None

def g_to_p_phonographeme_consistency_distribution_quantile(word, q): # e.g. q = 0.25 for Q1, q = 0.7 for 70th percentile
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0 and q >= 0 and q <= 100:
        return np.quantile(g_to_p_probs, q)
    return None

def g_to_p_phonographeme_consistency_IQR(word):
    return g_to_p_phonographeme_consistency_distribution_quantile(word, 0.75) - g_to_p_phonographeme_consistency_distribution_quantile(word, 0.25)

def g_to_p_phonographeme_consistency_range(word):
    return g_to_p_phonographeme_consistency_max(word) - g_to_p_phonographeme_consistency_min(word)

In [200]:
word_input = input("G->P PHONOGRAPHEME CONSISTENCY\nEnter word:\n")
if word_input.lower() not in df['STRING'].values:
    print("Word not found. Exiting.")
else:
    print("MEDIAN:", g_to_p_phonographeme_consistency_median(word_input))
    print("MEAN:", g_to_p_phonographeme_consistency_mean(word_input))
    print("MAX:", g_to_p_phonographeme_consistency_max(word_input))
    print("MIN:", g_to_p_phonographeme_consistency_min(word_input))
    print("Q1:", g_to_p_phonographeme_consistency_distribution_quantile(word_input, 0.7)) # changed this so its not currently Q1
    print("IQR:", g_to_p_phonographeme_consistency_IQR(word_input))
    print("RANGE:", g_to_p_phonographeme_consistency_range(word_input))

    # 0.25 - 0.5 range? not just 0.7 and lower etc etc

KeyboardInterrupt: Interrupted by user

In [ ]:
# # ONSET P->G PHONOGRAPHEME CONSISTENCY

# def onset_calculate_p_to_g_phonographeme_probs(onset):
#     onset = onset.lower()
#     p_to_g_probs = []
#     if onset in df['onset_g'].values:
#         word_phonemes = [df.loc[(df['onset_g'] == onset), 'initial1'].values[0], df.loc[(df['STRING'] == word), '1a'].values[0], df.loc[(df['STRING'] == word), '1b'].values[0]]
#         word_phonemes = [x for x in word_phonemes if str(x) != 'nan']
#         word_graphemes = [df.loc[(df['STRING'] == word), 'initial1.1'].values[0], df.loc[(df['STRING'] == word), '1a.1'].values[0], df.loc[(df['STRING'] == word), '1b.1'].values[0]]
#         word_graphemes = [x for x in word_graphemes if str(x) != 'nan']
#         for index in range(len(word_phonemes)):
#             p_to_g_probs.append(conditional_prob_p_to_g(word_phonemes[index], word_graphemes[index]))
#     return p_to_g_probs
    
# def onset_p_to_g_phonographeme_consistency_median(word):
#     p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
#     if len(p_to_g_probs) != 0:
#         return statistics.median(p_to_g_probs)
#     return None

# def onset_p_to_g_phonographeme_consistency_mean(word):
#     p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
#     if len(p_to_g_probs) != 0:
#         return statistics.mean(p_to_g_probs)
#     return None

# def onset_p_to_g_phonographeme_consistency_max(word):
#     p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
#     if len(p_to_g_probs) != 0:
#         return max(p_to_g_probs)
#     return None

# def onset_p_to_g_phonographeme_consistency_min(word):
#     p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
#     if len(p_to_g_probs) != 0:
#         return min(p_to_g_probs)
#     return None

# def onset_p_to_g_phonographeme_consistency_distribution_quantile(word, q): # e.g. q = 0.25 for Q1, q = 0.7 for 70th percentile
#     p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
#     if len(p_to_g_probs) != 0 and q >= 0 and q <= 100:
#         return np.quantile(p_to_g_probs, q)
#     return None

# def onset_p_to_g_phonographeme_consistency_IQR(word):
#     return p_to_g_phonographeme_consistency_distribution_quantile(word, 0.75) - p_to_g_phonographeme_consistency_distribution_quantile(word, 0.25)

# def onset_p_to_g_phonographeme_consistency_range(word):
#     return onset_p_to_g_phonographeme_consistency_max(word) - onset_p_to_g_phonographeme_consistency_min(word)

In [201]:
df.to_csv('dataframe.csv') # relative position